In [2]:
# The email scrape filters for 12/16/2023 and on


%load_ext autoreload
%autoreload 2
# import importlib
# importlib.reload()

import logging
import pandas as pd
from config import imap_password_customplanet, db_username, db_password
from modules.scrapes import scrape
from modules.db_operations import DatabaseConnector

email_address = 'team@customplanet.com'
email_pass = imap_password_customplanet


logging.basicConfig(filename='Email_Scraper.log', level=logging.INFO,
                   format='%(asctime)s - %(message)s', datefmt='%d-%b-%y %H:%M:%S',force=True)


# ------------------------------------packaging process into final function----------------------------------------

def process(subject_line, email_address, email_pass, start_date):

    logging.info('\n\nNew logging instance')

    #create the msgs via scrape, then iterate through list of messages, and lastly cleanse frame
    email_address = 'team@customplanet.com'
    email_pass = imap_password_customplanet
 
    msgs_inbox = scrape.scrape_msgs_outbox_or_inbox('inbox', subject_line, email_address, email_pass, start_date)
    inbox = scrape.create_msg_frame(msgs_inbox)
    inbox = scrape.cleanse_frame(inbox, 'inbox')
    inbox.name = 'inbox'

    msgs_outbox = scrape.scrape_msgs_outbox_or_inbox('outbox', subject_line, email_address, email_pass, start_date)
    outbox = scrape.create_msg_frame(msgs_outbox)
    outbox = scrape.cleanse_frame(outbox, 'outbox')
    outbox.name = 'outbox'
    
    if outbox.empty == True:
        thread = None
        pass
    else: #this is a left merge on outbox
        thread = scrape.piece_together(outbox, inbox)
        thread = scrape.assign_sentiment(thread)
    
    # Specify your database connection details
    server = 'emailcampaign.c9vhoi6ncot7.us-east-1.rds.amazonaws.com'
    database = 'emailcampaign'
    db = 'emailcampaign'
    table_name = 'thread'

    # Instantiate the DatabaseConnector class
    db_connector = DatabaseConnector(server, database, db_username, db_password, db, table_name)

    #If there are any new records, append them onto thread table
    #Update the reply_thread everytime
    if thread.empty == False:
        new_records = db_connector.append_new_records(thread)
        db_connector.update_reply_thread(thread)
        logging.info(f'Thread has been updated with {len(new_records)}')
    else:
        logging.info('Thread is none')
        print('Thread is none')
        new_records = None

    # return(inbox, outbox, thread, new_records)
    return(inbox, outbox, thread, new_records)

# The foreign constraint enforces that this must have been sent initially. 
inbox, outbox, thread, new_records  = process('Sport Special', email_address, email_pass, '11/15/2023')

#4285 emails in the inbox took 21 minutes

#Send out test emails for local baskebtall jsersyes
#Then send out for local football jerseys
# The scrape subject line, and email subject line should be referencing one variable

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
You have 3 email responses to the specified search criteria Sport Special in the inbox
You have 9 email responses to the specified search criteria Sport Special in the outbox


c:\Users\samuel.taylor\OneDrive - Green Dot Public Schools\Desktop\Git_Directory\CP\CustomPlanet_Work\Email_Scraper\Scraping_Emails\modules\db_operations.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return(df_SQL)
c:\Users\samuel.taylor\OneDrive - Green Dot Public Schools\Desktop\Git_Directory\CP\CustomPlanet_Work\Email_Scraper\Scraping_Emails\modules\db_operations.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return(df_SQL)
c:\Users\samuel.taylor\OneDrive - Green Dot Public Schools\Desktop\Git_Directory\CP\CustomPlanet_Work\Email_Scraper\Scraping_Emails\modules\db_operations.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or d

No new records to send
Reply_Thread updated
Reply_Thread updated
Reply_Thread updated


# To DO

In [ ]:
#Create PNGs on hoodies images using Selenium. 

#Write directly to MySQL DB

#Refine NLP

#Thirdly, what if the email is not directly replied to. In that case. 
# The scenarion would be the following, email campaign sent out to sammytaylor2006@yahoo.com
#The customer indirectly reaches out to CP, and not through the thread. This calls for a seperate table.
# and a seperate scrape 

#Integrate SSIS at some point for views
#Integrating prior orders and linking them in a SQL view.

#Predictive analytics based on prior orders

#Put the PNG on the given sport jersey
#Clean the CSV to iterate properly, this should be done. 
#Singularize the db_operations to reference in both